In [35]:
import pandas as pd
from ccl import  CCL
import torch, torchvision
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import ResNet as ResNet
from torch.utils.data import Dataset
from timm.models.layers.helpers import to_2tuple
import timm
import numpy as np

# retccl

In [10]:
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
trnsfrms_val = transforms.Compose(
    [
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean = mean, std = std)
    ]
)

In [ ]:
model = ResNet.resnet50(num_classes=128, mlp=False, two_branch=False, normlinear=True)
state_dict = torch.load(r'best_ckpt.pth')
model.fc = nn.Identity()
model.load_state_dict(pretext_model, strict=True)

model.cuda()

In [21]:
data_df = np.load(f'../../config/data_segmentation_csv/10X_full.npy',allow_pickle=True).item()['full_list']
path = data_df['path'][0][:-3]+'_array.npy'
image_array = np.load(path,allow_pickle=True)
img_seq = image_array[:10]
images = []
for p in img_seq:
    image = Image.fromarray(p[1]).convert('RGB')
    image_tensor = trnsfrms_val(image).unsqueeze(0)
    images.append(image_tensor)
images = torch.cat(images, dim=0)
images.shape

torch.Size([10, 3, 256, 256])

In [25]:
features = model(images.cuda()) 

In [26]:
features.shape

torch.Size([10, 2048])

# cTransPath

In [1]:
import pandas as pd
import torch, torchvision
import torch.nn as nn
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset
from timm.models.layers.helpers import to_2tuple
import timm

/GPUFS/sysu_jhluo_1/.conda/envs/ds/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /GPUFS/sysu_jhluo_1/.conda/envs/ds/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefINS2_6SymIntEEENS2_8optionalINS2_10ScalarTypeEEENS6_INS2_6LayoutEEENS6_INS2_6DeviceEEENS6_IbEENS6_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
class ConvStem(nn.Module):

    def __init__(self, img_size=224, patch_size=4, in_chans=3, embed_dim=768, norm_layer=None, flatten=True):
        super().__init__()

        assert patch_size == 4
        assert embed_dim % 8 == 0

        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        self.img_size = img_size
        self.patch_size = patch_size
        self.grid_size = (img_size[0] // patch_size[0], img_size[1] // patch_size[1])
        self.num_patches = self.grid_size[0] * self.grid_size[1]
        self.flatten = flatten


        stem = []
        input_dim, output_dim = 3, embed_dim // 8
        for l in range(2):
            stem.append(nn.Conv2d(input_dim, output_dim, kernel_size=3, stride=2, padding=1, bias=False))
            stem.append(nn.BatchNorm2d(output_dim))
            stem.append(nn.ReLU(inplace=True))
            input_dim = output_dim
            output_dim *= 2
        stem.append(nn.Conv2d(input_dim, embed_dim, kernel_size=1))
        self.proj = nn.Sequential(*stem)

        self.norm = norm_layer(embed_dim) if norm_layer else nn.Identity()

    def forward(self, x):
        B, C, H, W = x.shape
        assert H == self.img_size[0] and W == self.img_size[1], \
            f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})."
        x = self.proj(x)
        if self.flatten:
            x = x.flatten(2).transpose(1, 2)  # BCHW -> BNC
        x = self.norm(x)
        return x

def ctranspath():
    model = timm.create_model('swin_tiny_patch4_window7_224', embed_layer=ConvStem, pretrained=False)
    return model

In [13]:
model = ctranspath()
model.head = nn.Identity()
td = torch.load(r'ctranspath.pth')
model.load_state_dict(td['model'], strict=True)
model.cuda()

SwinTransformer(
  (patch_embed): ConvStem(
    (proj): Sequential(
      (0): Conv2d(3, 12, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(12, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (4): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): Conv2d(24, 96, kernel_size=(1, 1), stride=(1, 1))
    )
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (layers): Sequential(
    (0): BasicLayer(
      dim=96, input_resolution=(56, 56), depth=2
      (blocks): ModuleList(
        (0): SwinTransformerBlock(
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            (qkv): Linear(in_features=96, out_features=288, bias=True

In [5]:
import numpy as np
import pandas as pd

In [14]:
data_df = np.load(f'../../config/data_segmentation_csv/10X_full.npy',allow_pickle=True).item()['full_list']
path = data_df['path'][0][:-3]+'_array.npy'
image_array = np.load(path,allow_pickle=True)
img_seq = image_array[:10]
images = []
for p in img_seq:
    image = Image.fromarray(p[1]).convert('RGB')
    image_tensor = trnsfrms_val(image).unsqueeze(0)
    images.append(image_tensor)
images = torch.cat(images, dim=0)
images.shape

torch.Size([10, 3, 224, 224])

In [15]:
features = model(images.cuda()) 
features.shape

torch.Size([10, 768])